In [1]:
# https://huggingface.co/prajjwal1/bert-small
!pip install datasets
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [3]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

# An example code to do that

In [4]:
#import torch
from transformers import AutoTokenizer, AutoModel

device = "cuda" # the device to load the model onto
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-small')
model = AutoModel.from_pretrained("prajjwal1/bert-small")

text = "Replace me by any text you'd like."

encoded_input = tokenizer(text, return_tensors='pt').to(device)
model.to(device)

output = model(**encoded_input)

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
pd.Series(dataset['train'][:]['label']).value_counts()

In [ ]:
X_train = dataset['train'][:]['text'].copy()
y_train = dataset['train'][:]['label'].copy()
X_val = dataset['validation'][:]['text'].copy()
y_val = dataset['validation'][:]['label'].copy()
X_test = dataset['test'][:]['text'].copy()
y_test = dataset['test'][:]['label'].copy()

In [ ]:
X_all = np.concatenate((X_train, X_val, X_test), axis = 0)
X_all.shape

In [ ]:
y_all = np.concatenate((y_train, y_val, y_test), axis = 0)
y_all.shape

In [ ]:
unique_values, counts = np.unique(y_all, return_counts=True)
value_counts = dict(zip(unique_values, counts))
print("Value Counts:")
for value, count in value_counts.items():
    print(f"{value}: {count}")

In [ ]:
data = pd.DataFrame({
    'category': y_all,
    'text': X_all
})
data.head()

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(data.sample(frac=1, random_state=42),
                                     [int(.8*len(data)), int(.9*len(data))])

print(len(df_train),len(df_val), len(df_test))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

def extract_features(model, tokenizer, text, device="cuda"):
    # Tokenize the input text and move it to the specified device
    encoded_input = tokenizer(text, return_tensors='pt').to(device)

    # Forward pass through the model to obtain the hidden states
    with torch.no_grad():
        output = model(**encoded_input)

    # Extract the hidden states (output[0]) from the output
    hidden_states = output.last_hidden_state

    # For example, you can extract the representation at the [CLS] token (first token)
    cls_representation = hidden_states[:, 0, :]

    return cls_representation

# Example usage:
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-small')
model = AutoModel.from_pretrained("prajjwal1/bert-small").to(device)

text = "Replace me by any text you'd like."

features = extract_features(model, tokenizer, text, device)

In [ ]:
train_features, train_labels = extract_features(df_train)
test_features, test_labels = extract_features(da_test)
val_features, val_labels = extract_features(df_val)